In [62]:
%%capture
%run 05_block.ipynb
%run 06_mine.ipynb

Account state holds the current balance and nonce of an account.

In [63]:
class AccountState:
    def __init__(self): self.balance,self.nonce = 0,0
    def inc_nonce  (self):        self.nonce   += 1
    def add_balance(self, value): self.balance += value
    def sub_balance(self, value): self.balance -= value
    def __str__(self): return f'balance: {self.balance}\nnonce: {self.nonce}'

The state stores the dictionary `state` which stores the `AccountState` for each address. A genesis block is used to initialize state the first time.

The state is updated with txs. `apply` validates if the tx is correct and then updates each `AccountState` accordingly.

In [183]:
class State:
    def __init__(self): self.state={}
    def new   (self, pub): self.state[pub] = AccountState()
    def is_new(self, pub): return pub not in self.state
    def add(self, pubs):
        for pub in pubs:
            if self.is_new(pub): self.new(pub)
    
    def init(self, gb):
        gh = gb.header
        assert gh.number    == 0
        assert gh.prev_hash == '0x0'
        for tx in gb.txs.values():
            self.add([tx.fr, tx.to])
            assert val_sig(tx)
            self.state[tx.fr].inc_nonce()
            self.state[tx.to].add_balance(tx.value)
        
    def apply(self, tx, miner):
        self.add([tx.fr,tx.to,miner])
        if tx.fr != tx.to: # if coinbase tx
            assert val_sig(tx)
            assert self.state[tx.fr].nonce  == tx.nonce
            assert self.state[tx.fr].balance - tx.value > 0
        self.state[tx.fr].inc_nonce()
        self.state[tx.fr].sub_balance(tx.value)
        self.state[tx.to].add_balance(tx.value)
        self.state[miner].add_balance(tx.fee)
        return True
    
    def apply_reward(self, miner, reward):
        self.state[miner].add_balance(reward)
            
    def __str__(self):
        return '\n'.join(f'{ph(k)}\n{v}\n' for k,v in self.state.items())

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by its `prev_hash`.

A `candidate` block is one that could be used for mining.

In [266]:
class Blockchain:
    def __init__(self, gb):
        self.state  = State()
        self.blocks = []
        self.state.init(gb)
        self.blocks.append(gb)

    def candidate(self, txs):
        mt = MerkleTree(txs)
        bh = Header(mt.root, self.blocks[-1].hash, len(self.blocks), len(txs))
        return Block(bh, txs)
        
    def add(self, mb):
        assert self.val(mb)
        for tx in mb.txs.values(): assert self.state.apply(tx, mb.header.miner)
        self.state.apply_reward(mb.header.miner, mb.header.reward)
        self.blocks.append(mb)
        return True
    
    def val(self, mb):
        bh = mb.header
        assert val_pow(mb)
        assert bh.number == len(self.blocks)
        if len(self.blocks) > 0: assert self.blocks[-1].hash == bh.prev_hash
        return True
    
    def __str__(self): 
        return (('\n'*2+'--'*20+'\n'*2).join(f'{str(block)}' for block in self.blocks)
                +'\n'*2+'-'*6+'\nstate:\n'+'-'*6
                +'\n'+str(self.state))

In [267]:
acc1,acc2,acc3 = Account(),Account(),Account()
miner = Miner(acc3)

In [268]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

gb = mine_genesis([tx1, tx2])

In [269]:
bc = Blockchain(gb); print(bc)

root:          💴 0x4ee4007fea...2f1
hash:          🕏 0xe9e35c23d7...795
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          Wed Mar 31 01:38:57 2021
volume:        42.0 ether
fees:          0.22 ether
sz:            2374

txs:
💃 0x1d642cf334...b85
💨 0x427fa02cb1...61b

------
state:
------
👿 0x190e1c2209...345
balance: 18.0
nonce: 1

💲 0x4cC303CB4A...315
balance: 24.0
nonce: 1



In [270]:
tx1 = acc1.signed_tx(acc2, 3, 0.05)
tx2 = acc2.signed_tx(acc1, 9, 0.16)

block = bc.candidate([tx1,tx2])
mb    = miner.mine([tx1,tx2], gb.header, 2, 50)
bc.add(mb); print(bc)

root:          💴 0x4ee4007fea...2f1
hash:          🕏 0xe9e35c23d7...795
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          Wed Mar 31 01:38:57 2021
volume:        42.0 ether
fees:          0.22 ether
sz:            2374

txs:
💃 0x1d642cf334...b85
💨 0x427fa02cb1...61b

----------------------------------------

root:          👨 0x0279923580...f73
hash:          🔙 0xb30f58d1bb...65b
prev_hash:     🕏 0xe9e35c23d7...795
number:        1
n_txs:         3
mined:         True
time:          Wed Mar 31 01:39:03 2021
diff:          2
reward:        50
miner:         🔭 0xC7e4Bd59Fc...1AC
nonce:         123
volume:        62.0 ether
fees:          0.21 ether
sz:            3507

txs:
👹 0x1347c583f4...91c
📖 0x7075c4069b...d0a
📱 0x8b0ff6f98f...2c9

------
state:
------
👿 0x190e1c2209...345
balance: 24.0
nonce: 2

💲 0x4cC303CB4A...315
balance: 18.0
nonce: 2

🔭 0xC7e4Bd59Fc...1AC
balance: 50.21
nonce: 1



In [271]:
acc4 = Account()
tx1 = acc2.signed_tx(acc4, 12, 1.5)

block = bc.candidate([tx1])
mb    = miner.mine([tx1], mb.header, 2, 50)
bc.add(mb); print(bc)

root:          💴 0x4ee4007fea...2f1
hash:          🕏 0xe9e35c23d7...795
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          Wed Mar 31 01:38:57 2021
volume:        42.0 ether
fees:          0.22 ether
sz:            2374

txs:
💃 0x1d642cf334...b85
💨 0x427fa02cb1...61b

----------------------------------------

root:          👨 0x0279923580...f73
hash:          🔙 0xb30f58d1bb...65b
prev_hash:     🕏 0xe9e35c23d7...795
number:        1
n_txs:         3
mined:         True
time:          Wed Mar 31 01:39:03 2021
diff:          2
reward:        50
miner:         🔭 0xC7e4Bd59Fc...1AC
nonce:         123
volume:        62.0 ether
fees:          0.21 ether
sz:            3507

txs:
👹 0x1347c583f4...91c
📖 0x7075c4069b...d0a
📱 0x8b0ff6f98f...2c9

----------------------------------------

root:          💷 0x5121507263...e1e
hash:          🕤 0xfeb473a9bb...73d
prev_hash:     🔙 0xb30f58d1bb...65b
number:        2
n_txs:         2
mined:         True
time:        